# Tweet Cleaning

### Import Libraries

In [489]:
import numpy as np
import pandas as pd
import os

# Word processing libraries
import re
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from shapely.geometry import LineString
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
import folium

#Nan
from cmath import nan

### Read Tweets from CSV - Provided by scraping

In [490]:
tweets = pd.read_csv('CSV/Scrape_Tweets.csv')

In [491]:
tweets.shape

(128910, 13)

In [492]:
tweets = tweets.drop(['Unnamed: 0'], axis=1)
tweets.head()

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,1575636537294737408,1005713558955610112,3276,@NduLindani Thank you I’ll go to the doctor ag...,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]"
1,1575636504185274369,343624915,138,@RiebvJanbeeck @marcel__za Always good,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195..."
2,1575636335779794944,1135662061416062977,730,Watch full vlog on YouTube \n\nhttps://t.co/ro...,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396..."
3,1575636217365872640,1234830631,2515,You're a joke. https://t.co/Tfq5N7t4Mr,2022-09-29 23:58:42+00:00,0,0,0,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681..."
4,1575636196180389889,1353457542052143107,1323,@dipam @ufcfooty Highly doubt it.,2022-09-29 23:58:37+00:00,0,0,0,0,{'place_id': 'cc95b56a28712044'},"Centurion, South Africa","[28.0010585, -25.9448996, 28.2712217, -25.7768..."


In [493]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128910 entries, 0 to 128909
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   tweet_id          128910 non-null  int64 
 1   author_id         128910 non-null  int64 
 2   author_followers  128910 non-null  int64 
 3   text              128910 non-null  object
 4   created_at        128910 non-null  object
 5   retweets          128910 non-null  int64 
 6   replies           128910 non-null  int64 
 7   likes             128910 non-null  int64 
 8   quote_count       128910 non-null  int64 
 9   place_id          128910 non-null  object
 10  place_name        128910 non-null  object
 11  bbox              128910 non-null  object
dtypes: int64(7), object(5)
memory usage: 11.8+ MB


### Remove Duplicates
If entry is the same then drop it</br>
There should't be duplicates

In [494]:
print('Initial size of dataset before dropping duplicated rows:', tweets.shape)
tweets.drop_duplicates(keep = False, inplace = True)

print('Current size of dataset after dropping duplicated rows, if any, is:', tweets.shape)

Initial size of dataset before dropping duplicated rows: (128910, 12)
Current size of dataset after dropping duplicated rows, if any, is: (128910, 12)


### Remove Empty Tweets
If tweet content is empty/Nan then drop it

In [495]:
tweets.dropna(subset = ['text'], inplace = True)

In [496]:
len(tweets)

128910

### Collect @Users in Text
Identify all mentions of other users using @ </br>
Create new feature containg all mentions (@s)</br>
Remove all mentions from text - done in next section

In [497]:
def mentioned_users(string):
    usernames = re.findall('@[^\s]+', string)
    if usernames == []:
        return nan
    return usernames

In [498]:
tweets['mentioned_users'] = tweets['text'].apply(lambda x: mentioned_users(x))
tweets.head()

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox,mentioned_users
0,1575636537294737408,1005713558955610112,3276,@NduLindani Thank you I’ll go to the doctor ag...,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]",[@NduLindani]
1,1575636504185274369,343624915,138,@RiebvJanbeeck @marcel__za Always good,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195...","[@RiebvJanbeeck, @marcel__za]"
2,1575636335779794944,1135662061416062977,730,Watch full vlog on YouTube \n\nhttps://t.co/ro...,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",NaN
3,1575636217365872640,1234830631,2515,You're a joke. https://t.co/Tfq5N7t4Mr,2022-09-29 23:58:42+00:00,0,0,0,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681...",NaN
4,1575636196180389889,1353457542052143107,1323,@dipam @ufcfooty Highly doubt it.,2022-09-29 23:58:37+00:00,0,0,0,0,{'place_id': 'cc95b56a28712044'},"Centurion, South Africa","[28.0010585, -25.9448996, 28.2712217, -25.7768...","[@dipam, @ufcfooty]"


### Collect #Hashtags in Text
Identify all hashtags using # </br>
Create new feature containg all hashtags (#s)</br>
Remove all hashtags from text - done in next section

In [499]:
def hashtags(string):
    hashtags = re.findall('#[^\s]+', string)
    if hashtags == []:
        return nan
    return hashtags

In [500]:
tweets['hashtags'] = tweets['text'].apply(lambda x: hashtags(x))
tweets.head()

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox,mentioned_users,hashtags
0,1575636537294737408,1005713558955610112,3276,@NduLindani Thank you I’ll go to the doctor ag...,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]",[@NduLindani],NaN
1,1575636504185274369,343624915,138,@RiebvJanbeeck @marcel__za Always good,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195...","[@RiebvJanbeeck, @marcel__za]",NaN
2,1575636335779794944,1135662061416062977,730,Watch full vlog on YouTube \n\nhttps://t.co/ro...,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",NaN,NaN
3,1575636217365872640,1234830631,2515,You're a joke. https://t.co/Tfq5N7t4Mr,2022-09-29 23:58:42+00:00,0,0,0,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681...",NaN,NaN
4,1575636196180389889,1353457542052143107,1323,@dipam @ufcfooty Highly doubt it.,2022-09-29 23:58:37+00:00,0,0,0,0,{'place_id': 'cc95b56a28712044'},"Centurion, South Africa","[28.0010585, -25.9448996, 28.2712217, -25.7768...","[@dipam, @ufcfooty]",NaN


### Collect Emojis in text
Identify all emojis using unicode value</br>
Create new feature containg all emojis</br>
Remove all emojis from text - done in next section</br>
Note: We could identify our own emojis that could be useful instead of all emojis and put them in a dictionary

In [501]:
from cmath import nan
import advertools as adv
def extract_emojis(string):
    list = [string]
    emoji_dict = adv.extract_emoji(list)
    emojis = emoji_dict['emoji'][0]
    if(emojis == []):
        return nan
    return emojis

In [502]:
# For if an error saying float can not be changed to lower is called!
#tweets.text=tweets.text.astype(str)

In [503]:
tweets['emojis'] = tweets['text'].apply(lambda x: extract_emojis(x))
tweets.head()

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox,mentioned_users,hashtags,emojis
0,1575636537294737408,1005713558955610112,3276,@NduLindani Thank you I’ll go to the doctor ag...,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]",[@NduLindani],NaN,NaN
1,1575636504185274369,343624915,138,@RiebvJanbeeck @marcel__za Always good,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195...","[@RiebvJanbeeck, @marcel__za]",NaN,NaN
2,1575636335779794944,1135662061416062977,730,Watch full vlog on YouTube \n\nhttps://t.co/ro...,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",NaN,NaN,NaN
3,1575636217365872640,1234830631,2515,You're a joke. https://t.co/Tfq5N7t4Mr,2022-09-29 23:58:42+00:00,0,0,0,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681...",NaN,NaN,NaN
4,1575636196180389889,1353457542052143107,1323,@dipam @ufcfooty Highly doubt it.,2022-09-29 23:58:37+00:00,0,0,0,0,{'place_id': 'cc95b56a28712044'},"Centurion, South Africa","[28.0010585, -25.9448996, 28.2712217, -25.7768...","[@dipam, @ufcfooty]",NaN,NaN


### Collect Links in text
Identify all links using a URL</br>
Create new feature containg all Links</br>
Remove all links from text - done in next section</br>
Maybe look into if 'www' syntax must also be used

In [504]:
def find_urls(string):
    try:
        urls = re.search("(?P<url>https?://[^\s]+)", string).group("url")
    except:
        return nan
    return urls

In [505]:
tweets['urls'] = tweets['text'].apply(lambda x: find_urls(x))
tweets.head()

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox,mentioned_users,hashtags,emojis,urls
0,1575636537294737408,1005713558955610112,3276,@NduLindani Thank you I’ll go to the doctor ag...,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]",[@NduLindani],NaN,NaN,NaN
1,1575636504185274369,343624915,138,@RiebvJanbeeck @marcel__za Always good,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195...","[@RiebvJanbeeck, @marcel__za]",NaN,NaN,NaN
2,1575636335779794944,1135662061416062977,730,Watch full vlog on YouTube \n\nhttps://t.co/ro...,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",NaN,NaN,NaN,https://t.co/roAWcrHuvY
3,1575636217365872640,1234830631,2515,You're a joke. https://t.co/Tfq5N7t4Mr,2022-09-29 23:58:42+00:00,0,0,0,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681...",NaN,NaN,NaN,https://t.co/Tfq5N7t4Mr
4,1575636196180389889,1353457542052143107,1323,@dipam @ufcfooty Highly doubt it.,2022-09-29 23:58:37+00:00,0,0,0,0,{'place_id': 'cc95b56a28712044'},"Centurion, South Africa","[28.0010585, -25.9448996, 28.2712217, -25.7768...","[@dipam, @ufcfooty]",NaN,NaN,NaN


In [506]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128910 entries, 0 to 128909
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   tweet_id          128910 non-null  int64 
 1   author_id         128910 non-null  int64 
 2   author_followers  128910 non-null  int64 
 3   text              128910 non-null  object
 4   created_at        128910 non-null  object
 5   retweets          128910 non-null  int64 
 6   replies           128910 non-null  int64 
 7   likes             128910 non-null  int64 
 8   quote_count       128910 non-null  int64 
 9   place_id          128910 non-null  object
 10  place_name        128910 non-null  object
 11  bbox              128910 non-null  object
 12  mentioned_users   71871 non-null   object
 13  hashtags          12903 non-null   object
 14  emojis            52564 non-null   object
 15  urls              50703 non-null   object
dtypes: int64(7), object(9)
memory usage: 1

### Remove Unwanted Information and Clean Tweet text
To Clean Text:
* Convert to Lowercase
* Tokenise
* Tag Text
* Lemmatise Text

This includes:
* @mentions
* URLs
* Hashtags
* Emojis
* Punctuation
* Numbers
* Stop Words
* Single Letter Words
* Empty Tokens


In [507]:
# Define Emoji_patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [508]:
# Define the function to implement POS tagging:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


# Define the main function to clean text in various ways:
def clean_text(text):
    
    # Apply regex expressions first before converting string to list of tokens/words:
    # 1. remove @usernames
    text = re.sub('@[^\s]+', '', text)
    
    # 2. remove URLs
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    
    # 3. remove hashtags entirely i.e. #hashtags
    text = re.sub(r'#([^\s]+)', '', text)
    
    # 4. remove emojis
    text = emoji_pattern.sub(r'', text)
    
    # 5. Convert text to lowercase
    text = text.lower()
    
    # 6. tokenize text and remove punctuation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    
    # 7. remove numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    
    # 8. remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    
    # 9. remove empty tokens
    text = [t for t in text if len(t) > 0]
    
    # 10. pos tag text and lemmatize text
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    
    # 11. remove words with only one letter
    text = [t for t in text if len(t) > 1]
    
    # join all
    text = " ".join(text)
    
    return(text)

In [509]:
# Apply function on the column 'text':
tweets['cleaned_text'] = tweets['text'].apply(lambda x: clean_text(x))
tweets.head()

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox,mentioned_users,hashtags,emojis,urls,cleaned_text
0,1575636537294737408,1005713558955610112,3276,@NduLindani Thank you I’ll go to the doctor ag...,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]",[@NduLindani],NaN,NaN,NaN,thank i’ll go doctor ekuseni
1,1575636504185274369,343624915,138,@RiebvJanbeeck @marcel__za Always good,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195...","[@RiebvJanbeeck, @marcel__za]",NaN,NaN,NaN,always good
2,1575636335779794944,1135662061416062977,730,Watch full vlog on YouTube \n\nhttps://t.co/ro...,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",NaN,NaN,NaN,https://t.co/roAWcrHuvY,watch full vlog youtube \n\n
3,1575636217365872640,1234830631,2515,You're a joke. https://t.co/Tfq5N7t4Mr,2022-09-29 23:58:42+00:00,0,0,0,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681...",NaN,NaN,NaN,https://t.co/Tfq5N7t4Mr,joke
4,1575636196180389889,1353457542052143107,1323,@dipam @ufcfooty Highly doubt it.,2022-09-29 23:58:37+00:00,0,0,0,0,{'place_id': 'cc95b56a28712044'},"Centurion, South Africa","[28.0010585, -25.9448996, 28.2712217, -25.7768...","[@dipam, @ufcfooty]",NaN,NaN,NaN,highly doubt


We can now remove the original tweet text because the cleaned text is all that is needed.

In [510]:
tweets = tweets.drop('text', axis = 1)

In [511]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128910 entries, 0 to 128909
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   tweet_id          128910 non-null  int64 
 1   author_id         128910 non-null  int64 
 2   author_followers  128910 non-null  int64 
 3   created_at        128910 non-null  object
 4   retweets          128910 non-null  int64 
 5   replies           128910 non-null  int64 
 6   likes             128910 non-null  int64 
 7   quote_count       128910 non-null  int64 
 8   place_id          128910 non-null  object
 9   place_name        128910 non-null  object
 10  bbox              128910 non-null  object
 11  mentioned_users   71871 non-null   object
 12  hashtags          12903 non-null   object
 13  emojis            52564 non-null   object
 14  urls              50703 non-null   object
 15  cleaned_text      128910 non-null  object
dtypes: int64(7), object(9)
memory usage: 1

In [512]:
tweets.to_csv('CSV/MISC/temp.csv')

In [513]:
tweets = pd.read_csv('CSV/MISC/temp.csv')

### Convert Boundry Box to a set of coordinates of Latitude and Longitude
There are a few ways of doing this
* Take an average and find the middle of the Boundry Box
* Indentify where the location is using another API based on place_name
* Keep the location as a polygon and then place each user into a munucipality in hich the area is largest

The way we will do it here is using centroid of the boundry box

In [514]:
def bbox_to_coords(bbox):
    all_coords = bbox.split()
    coords = []
    for coordinate in all_coords:
        coordinate = coordinate.replace('[','')
        coordinate = coordinate.replace(']','')
        coordinate = coordinate.replace(',','')
        coord = float(coordinate)
        coords.append(coord)

    return coords

In [515]:
tweets['coords'] = tweets['bbox'].apply(lambda x: bbox_to_coords(x))

In [516]:
tweets['longitude_1'] = tweets['coords'].apply(lambda x: x[0])
tweets['latitude_1'] = tweets['coords'].apply(lambda x: x[1])
tweets['longitude_2'] = tweets['coords'].apply(lambda x: x[2])
tweets['latitude_2'] = tweets['coords'].apply(lambda x: x[3])

In [517]:
tweets.head(3)

,Unnamed: 0,tweet_id,author_id,author_followers,created_at,retweets,replies,likes,quote_count,place_id,...,mentioned_users,hashtags,emojis,urls,cleaned_text,coords,longitude_1,latitude_1,longitude_2,latitude_2
0,0,1575636537294737408,1005713558955610112,3276,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},...,['@NduLindani'],NaN,NaN,NaN,thank i’ll go doctor ekuseni,"[30.8789097, -30.055938, 31.0662492, -29.7453336]",30.878910,-30.055938,31.066249,-29.745334
1,1,1575636504185274369,343624915,138,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},...,"['@RiebvJanbeeck', '@marcel__za']",NaN,NaN,NaN,always good,"[28.0140761, -26.0572574, 28.1930256, -25.9195...",28.014076,-26.057257,28.193026,-25.919551
2,2,1575636335779794944,1135662061416062977,730,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},...,NaN,NaN,NaN,https://t.co/roAWcrHuvY,watch full vlog youtube \n\n,"[27.7518557, -26.5126489, 28.1843404, -26.0396...",27.751856,-26.512649,28.184340,-26.039628


In [518]:
tweets = tweets.drop('bbox', axis=1)

In [519]:
def find_centroid(coords):
    geometry = LineString([(coords[0], coords[1]),(coords[2], coords[3])])
    centroid = geometry.centroid
    return centroid

In [520]:
tweets['centroid'] = tweets['coords'].apply(lambda x: find_centroid(x))

In [521]:
tweets['centroid_long'] = tweets['centroid'].apply(lambda z: z.x)
tweets['centroid_lat'] = tweets['centroid'].apply(lambda z: z.y)

In [522]:
tweets = tweets.drop('centroid', axis =1)
tweets.head(3)

,Unnamed: 0,tweet_id,author_id,author_followers,created_at,retweets,replies,likes,quote_count,place_id,...,emojis,urls,cleaned_text,coords,longitude_1,latitude_1,longitude_2,latitude_2,centroid_long,centroid_lat
0,0,1575636537294737408,1005713558955610112,3276,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},...,NaN,NaN,thank i’ll go doctor ekuseni,"[30.8789097, -30.055938, 31.0662492, -29.7453336]",30.878910,-30.055938,31.066249,-29.745334,30.972579,-29.900636
1,1,1575636504185274369,343624915,138,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},...,NaN,NaN,always good,"[28.0140761, -26.0572574, 28.1930256, -25.9195...",28.014076,-26.057257,28.193026,-25.919551,28.103551,-25.988404
2,2,1575636335779794944,1135662061416062977,730,2022-09-29 23:59:11+00:00,0,0,0,0,{'place_id': 'e564d30dc173d2a8'},...,NaN,https://t.co/roAWcrHuvY,watch full vlog youtube \n\n,"[27.7518557, -26.5126489, 28.1843404, -26.0396...",27.751856,-26.512649,28.184340,-26.039628,27.968098,-26.276139


### Fix Edge Cases
* Cape Town
* Betty's Bay
* Bloubergstrand
* Mdumbi Beach 
</br>
Using: https://www.distancesto.com/coordinates/za/bloubergstrand-latitude-longitude/history/76385.html

In [523]:
tweets.loc[tweets.place_name == 'Cape Town, South Africa', ['centroid_long', 'centroid_lat']] = 18.4241, -33.9249
tweets.loc[tweets.place_name == 'Mdumbi Beach', ['centroid_long', 'centroid_lat']] = 29.215369, -31.933896
tweets.loc[tweets.place_name == "Betty's Bay, South Africa", ['centroid_long', 'centroid_lat']] = 18.92051, -34.34747
tweets.loc[tweets.place_name == 'Bloubergstrand', ['centroid_long', 'centroid_lat']] = 18.46173, -33.800418

In [524]:
geometry = [Point(xy) for xy in zip(tweets['centroid_long'], tweets['centroid_lat'])]
gdf = GeoDataFrame(tweets, geometry=geometry)  

In [525]:
tweets = tweets.drop('longitude_1', axis=1)
tweets = tweets.drop('longitude_2', axis=1)
tweets = tweets.drop('latitude_1', axis=1)
tweets = tweets.drop('latitude_2', axis=1)
tweets = tweets.drop('centroid_long', axis=1)
tweets = tweets.drop('centroid_lat', axis=1)
tweets.head(2)

,Unnamed: 0,tweet_id,author_id,author_followers,created_at,retweets,replies,likes,quote_count,place_id,place_name,mentioned_users,hashtags,emojis,urls,cleaned_text,coords,geometry
0,0,1575636537294737408,1005713558955610112,3276,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa",['@NduLindani'],NaN,NaN,NaN,thank i’ll go doctor ekuseni,"[30.8789097, -30.055938, 31.0662492, -29.7453336]",POINT (30.97258 -29.90064)
1,1,1575636504185274369,343624915,138,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","['@RiebvJanbeeck', '@marcel__za']",NaN,NaN,NaN,always good,"[28.0140761, -26.0572574, 28.1930256, -25.9195...",POINT (28.10355 -25.98840)


### Remove Useless locations
Drop row where location is = South Africa, since this location is not specfic enough and offers little value.

In [526]:
tweets.shape

(128910, 18)

In [527]:
tweets = tweets[tweets['place_name'] != 'South Africa']

In [528]:
tweets.shape

(114847, 18)

In [529]:
tweets.head(2)

,Unnamed: 0,tweet_id,author_id,author_followers,created_at,retweets,replies,likes,quote_count,place_id,place_name,mentioned_users,hashtags,emojis,urls,cleaned_text,coords,geometry
0,0,1575636537294737408,1005713558955610112,3276,2022-09-29 23:59:59+00:00,0,1,0,0,{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa",['@NduLindani'],NaN,NaN,NaN,thank i’ll go doctor ekuseni,"[30.8789097, -30.055938, 31.0662492, -29.7453336]",POINT (30.97258 -29.90064)
1,1,1575636504185274369,343624915,138,2022-09-29 23:59:51+00:00,0,0,1,0,{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","['@RiebvJanbeeck', '@marcel__za']",NaN,NaN,NaN,always good,"[28.0140761, -26.0572574, 28.1930256, -25.9195...",POINT (28.10355 -25.98840)


In [530]:
tweets.to_csv('CSV/Clean_Tweets_All_Info.csv')

### Set up tweet dataframe into a normalised relational database
Use the following databases:
* Tweet Database - Contains all the information about each tweet
* Location Database - Contains all the information about each location
* Author Database - Contains all the information specfic to each author
* Relational Database - Contains the relations between all the above databases

#### Location Database

In [531]:
locations = GeoDataFrame()
locations['place_id'] = tweets['place_id']
locations['place_name'] = tweets['place_name']
locations['coords'] = tweets['coords']
locations['geometry'] = tweets['geometry']
print(locations.shape)
locations = locations.loc[locations.astype(str).drop_duplicates().index]
print(locations.shape)
locations = locations.set_index('place_id')
locations.head(3)


(114847, 4)
(3474, 4)


,place_name,coords,geometry
place_id,,,
{'place_id': '9d7cae88ff6a29f5'},"Durban, South Africa","[30.8789097, -30.055938, 31.0662492, -29.7453336]",POINT (30.97258 -29.90064)
{'place_id': '46c1b1ab24d7e11a'},"Midrand, South Africa","[28.0140761, -26.0572574, 28.1930256, -25.9195...",POINT (28.10355 -25.98840)
{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa","[27.7518557, -26.5126489, 28.1843404, -26.0396...",POINT (27.96810 -26.27614)


In [532]:
locations.shape

(3474, 3)

#### Tweet Database

In [533]:
just_tweets = tweets
just_tweets = just_tweets.drop('place_name', axis = 1)
just_tweets = just_tweets.drop('coords', axis = 1)
just_tweets = just_tweets.drop('geometry', axis = 1)
just_tweets = just_tweets.drop('place_id', axis = 1)
just_tweets = just_tweets.drop('author_id', axis = 1)
just_tweets = just_tweets.set_index('tweet_id')
just_tweets.head(3)

,Unnamed: 0,author_followers,created_at,retweets,replies,likes,quote_count,mentioned_users,hashtags,emojis,urls,cleaned_text
tweet_id,,,,,,,,,,,,
1575636537294737408,0,3276,2022-09-29 23:59:59+00:00,0,1,0,0,['@NduLindani'],NaN,NaN,NaN,thank i’ll go doctor ekuseni
1575636504185274369,1,138,2022-09-29 23:59:51+00:00,0,0,1,0,"['@RiebvJanbeeck', '@marcel__za']",NaN,NaN,NaN,always good
1575636335779794944,2,730,2022-09-29 23:59:11+00:00,0,0,0,0,NaN,NaN,NaN,https://t.co/roAWcrHuvY,watch full vlog youtube \n\n


In [534]:
just_tweets.shape

(114847, 12)

#### Relational Database

In [535]:
relations = pd.DataFrame()
relations['tweet_id'] = tweets['tweet_id']
relations['place_id'] = tweets['place_id']
relations = relations.loc[relations.astype(str).drop_duplicates().index]
relations = relations.set_index('tweet_id')
relations.head()

,place_id
tweet_id,
1575636537294737408,{'place_id': '9d7cae88ff6a29f5'}
1575636504185274369,{'place_id': '46c1b1ab24d7e11a'}
1575636335779794944,{'place_id': 'e564d30dc173d2a8'}
1575636217365872640,{'place_id': 'a02e6c261fa62b42'}
1575636196180389889,{'place_id': 'cc95b56a28712044'}


In [536]:
relations.shape

(114847, 1)

#### Create csv for each database

In [537]:
just_tweets.to_csv('CSV/Relational_Databases/just_tweets.csv')
locations.to_csv('CSV/Relational_Databases/locations.csv')
relations.to_csv('CSV/Relational_Databases/relations.csv')

### Possible Still to do Cleaning
ensure that there are no NaN values, this can be done by either creating a custom value possibly using an average or alike. </br>
Or filling in a value such as n/a indicating that no value is available or provided.